In [1]:
import pydicom
import os

In [29]:
import pydicom
import numpy as np

def resample_slice_thickness(dicom_file_path, new_thickness):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file_path)

    # Extract voxel data
    voxel_data = dicom_data.pixel_array

    # Calculate the current and new thickness ratios
    current_thickness = dicom_data.SliceThickness
    ratio = current_thickness / new_thickness

    # Resample the voxel data along the z-axis
    new_shape = (int(voxel_data.shape[0] * ratio), voxel_data.shape[1])
    resampled_voxel_data = np.zeros(new_shape, dtype=voxel_data.dtype)
    for i in range(new_shape[0]):
        slice_index = int(i / ratio)
        resampled_voxel_data[i, :] = voxel_data[slice_index, :]

    # Save the resampled slice to a new DICOM file
    resampled_dicom = dicom_data.copy()
    resampled_dicom.SliceThickness = new_thickness
    resampled_dicom.Rows, resampled_dicom.Columns = new_shape
    resampled_dicom.PixelData = resampled_voxel_data.tobytes()

    # resampled_file_path = dicom_file_path.replace('.dcm', f'_resampled_{new_thickness}mm.dcm')
    resampled_dicom.save_as(dicom_file_path)

    print("Resampled DICOM saved to:", dicom_file_path)

In [30]:
import pydicom

def change_pixel_spacing(dicom_file_path, new_pixel_spacing):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file_path)

    # Update Pixel Spacing
    dicom_data.PixelSpacing = new_pixel_spacing

    # Calculate new dimensions based on new pixel spacing
    new_rows = int(dicom_data.Rows * (dicom_data.PixelSpacing[0] / new_pixel_spacing[0]))
    new_columns = int(dicom_data.Columns * (dicom_data.PixelSpacing[1] / new_pixel_spacing[1]))

    # Update Rows and Columns
    dicom_data.Rows = new_rows
    dicom_data.Columns = new_columns

    # Update Slice Thickness if necessary
    if len(new_pixel_spacing) > 2:
        dicom_data.SliceThickness = new_pixel_spacing[2]

    # Save modified DICOM file
    # modified_file_path = dicom_file_path.replace('.dcm', '_modified.dcm')
    dicom_data.save_as(dicom_file_path)

    print("Pixel spacing modified and saved to:", dicom_file_path)


# new_pixel_spacing = [.78125, .78125] # in mm, for example, [new_row_spacing, new_column_spacing]

In [21]:
import pydicom
import numpy as np

def rescale_hu_values(dicom_file_path, new_hu_values):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file_path)

    # Extract voxel data
    voxel_data = dicom_data.pixel_array

    # Rescale HU values
    old_slope = dicom_data.RescaleSlope if hasattr(dicom_data, 'RescaleSlope') else 1.0
    old_intercept = dicom_data.RescaleIntercept if hasattr(dicom_data, 'RescaleIntercept') else 0.0

    rescaled_voxel_data = voxel_data * old_slope + old_intercept

    new_slope = (new_hu_values[1] - new_hu_values[0]) / (rescaled_voxel_data.max() - rescaled_voxel_data.min())
    new_intercept = new_hu_values[0] - new_slope * rescaled_voxel_data.min()

    rescaled_voxel_data = rescaled_voxel_data * new_slope + new_intercept

    # Update DICOM metadata if necessary
    dicom_data.RescaleIntercept = new_intercept
    dicom_data.RescaleSlope = new_slope

    # Save modified DICOM file
    # modified_file_path = dicom_file_path.replace('.dcm', '_rescaled.dcm')
    dicom_data.PixelData = rescaled_voxel_data.tobytes()
    dicom_data.save_as(dicom_file_path)

    print("HU values rescaled and saved to:", dicom_file_path)

# Example usage:
# dicom_file_path = "path_to_dicom_file.dcm"
# new_hu_values = (-1000, 1000)  # Example: clip HU values between -1000 and 1000

# rescale_hu_values(dicom_file_path, new_hu_values)

In [22]:
import os
path = os.path.join(os.getcwd(), "2013032205202300001")


for root,dir,filename in os.walk(path):
    for filenames in filename:
        ffile = os.path.join(root, filenames)
        resample_slice_thickness(ffile, new_thickness=1.24507868289948)
        change_pixel_spacing(ffile, new_pixel_spacing=[.78125, .78125])
        rescale_hu_values(ffile, new_hu_values = (-1000, 1000))

        ds = pydicom.dcmread(ffile)
        ds.SeriesDescription = "Changed slicer thickness, pixel spacing and rescale hu values"
        ds.save_as(ffile)
        # print(ds.PixelRepresentation)

c:\Python312\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: Invalid value for VR UI: '2013032205202300001.0.0.00.1684738133238'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


Resampled DICOM saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.0.dcm
Pixel spacing modified and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.0.dcm
HU values rescaled and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.0.dcm
Resampled DICOM saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.1.dcm
Pixel spacing modified and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.1.dcm
HU values rescaled and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.1.dcm
Resampled DICOM saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.10.dcm
Pixel spacing modified and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.10.dcm
HU values rescaled and saved to: d:\python\rtstruct read name\2013032205202300001\2013032205202300001.0.0.10.dc

In [39]:
import os
path = "C:\\Users\\91889\\Documents\\new lung data\\2013032205202300003"

for root,dir,filename in os.walk(path):
    for filenames in filename:
        ffile = os.path.join(root, filenames)
        resample_slice_thickness(ffile, new_thickness=1.24507868289948)
        change_pixel_spacing(ffile, new_pixel_spacing=[.78125, .78125])
        rescale_hu_values(ffile, new_hu_values = (-1000, 3071))

        # ds.SeriesDescription = "Changed slicer thickness, pixel spacing and rescale hu values"

        # ds = pydicom.dcmread(ffile)
        # ds.save_as(ffile)
        # print(ds.SliceThickness)

Resampled DICOM saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\2013032205202300003.0.1684738514917\2013032205202300003.0\2013032205202300003.0.0.0.dcm
Pixel spacing modified and saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\2013032205202300003.0.1684738514917\2013032205202300003.0\2013032205202300003.0.0.0.dcm
HU values rescaled and saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\2013032205202300003.0.1684738514917\2013032205202300003.0\2013032205202300003.0.0.0.dcm
Resampled DICOM saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\2013032205202300003.0.1684738514917\2013032205202300003.0\2013032205202300003.0.0.1.dcm
Pixel spacing modified and saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\2013032205202300003.0.1684738514917\2013032205202300003.0\2013032205202300003.0.0.1.dcm
HU values rescaled and saved to: C:\Users\91889\Documents\new lung data\2013032205202300003\20130322052023